In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# print("Num GPUs available: ", len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Data  Preparation

In [ ]:
#organizing data into specific folders and sub-folders

os.chdir('data/dogs-vs-cats')
# all images in this folder has images of dogs and cats labelled as 'cat.54.jpg', 'cat.655.jpg', 'dog.78.jpg', etc. 

if os.path.isdir('train/dog') is False:
    
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')
    
# we create training(1000), validation(200), test(100) data respectively
    
    for c in random.sample(glob.glob('cat*'), 500):
        shutil.move(c, 'train/cat')
    for c in random.sample(glob.glob('dog*'), 500):
        shutil.move(c, 'train/dog')
    for c in random.sample(glob.glob('cat*'), 100):
        shutil.move(c, 'valid/cat')
    for c in random.sample(glob.glob('dog*'), 100):
        shutil.move(c, 'valid/dog')
    for c in random.sample(glob.glob('cat*'), 50):
        shutil.move(c, 'test/cat')
    for c in random.sample(glob.glob('dog*'), 50):
        shutil.move(c, 'valid/dog')
        
os.chdir('../../')

In [ ]:
train_path = 'data/dogs-vs-cats/train'
valid_path = 'data/dogs-vs-cats/valid'
test_path = 'data/dogs-vs-cats/test'

### making data suitable to be passed to the 'fit' function

### Preprocessing data,  Adjusting height and width of image to (224x224)

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
                .flow_from_directory(directory=train_path, target_size=(224, 224), classes=['cat', 'dog'], batch_size=10)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
                .flow_from_directory(directory=valid_path, target_size=(224, 224), classes=['cat', 'dog'], batch_size=10)

test_batches  = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
                .flow_from_directory(directory=test_path, target_size=(224, 224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

In [ ]:
imgs, labels = next(train_batches)

#### This function will plot images in the form of a grid with 1 row and 10 columns where images are placed in first row and then labels

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes =axes.flatten()
    
    for img,ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show

In [ ]:
plotImages(imgs)
print(labels)

### Build and Train CNN

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

# here 'y' parameter is not needed in 'fit' function because we have generated images using 'ImageGenerator'
# and hence, its automatically incorporated there itself

### >>> If training accuracy is much greater than validation accuracy, it means OVERFITTING.

## Predictions

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
# Shuffling on 'test data' is not performed

test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=1)

In [ ]:
np.round(predictions)

### Creating CONFUSION MATRIX for better visualisation

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, 
                          title='Confusion Matrix', 
                          cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    
    else:
        print("Confusion matrix w/o normalization")
        
    print(cm)
    
    thres = cm.max()/2
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i,j], horizontalalignment="center", color="white" if cm[i, j]>thres else "black")
        
    plt.tight_layout()
    plt.ylabel('TRUE LABEL')
    plt.xlabel('PREDICTED LABEL')

In [ ]:
test_batches.class_indices

# decide the plotting of the 'confusion matrix'

In [ ]:
cm_plot_labels = ['CAT', 'DOG']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

### >> seeing the confusion matrix, one can be able to say how good his model is actually performing

##     

# Transfer Learning  /  Fine-Tuning the model

### Fine-Tuning VVG-16 model

In [ ]:
# download the pre-trained VGG-16 or preferred model architecture
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

In [ ]:
type(vgg16_model)

#### >>> In the VGG-16 architecture, we don't need the last layer as it classifies into '1000' classes whereas we want only 2 classes. Thus, we ignore the last layer.

In [ ]:
model = Sequential()

# ignoring the last layer by looping till the second-last layer
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [ ]:
model.summary()

### Adding the new desired layer (that will classify into 2 classes only)

#### (i) Freezing all the pretrained layers so that, they do not have to be retrained

In [ ]:
for layer in model.layers:
    layer.trainable = False

#### (ii) Adding the desired layer

In [ ]:
model.add( Dense(units=2, activation='softmax') )

In [ ]:
model.summary()

## Training the fine-tuned VGG-16 model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=2)

## Predictions using VGG-16 model

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=1)

#### >>>> Results clearly improved upon training on the fine-tuned model

### >>> Let's look at the CONFUSION MATRIX for better visualisation of our test accuracy

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = ['CAT', 'DOG']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

#### >>> we see better results here after fine-tuning